<a href="https://colab.research.google.com/github/peteryang/finetune_LLaVA/blob/main/llava_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!conda --version
#If !conda --version returns no results, install conda with :
!pip install -q condacolab
#import condacolab
#condacolab.install()

/bin/bash: line 1: conda: command not found


In [6]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [ ]:
!conda create -y -q -n llava python=3.10

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/llava

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    ld_impl_linux-64-2.40      |       h55db66e_0         697 KB  conda-forge
    libgcc-ng-13.2.0           |       h77fa898_7         758 KB  conda-forge
    libgomp-13.2.0             |       h77fa898_7         412 KB  conda-forge
    libsqlite-3.45.3           |       h2797004_0         840 KB  conda-forge
    libxcrypt-4.4.36           |       hd590300_1          98 KB  conda-forge
    ncurses-6.5                |       h59595ed_0         867 KB  conda-forge
    openssl-3.3.0              |       hd590300_0         2.8 MB  conda-forge
    pip-24.0                   |   

In [ ]:
!conda init

no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.10/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
no change     /root/.bashrc
No action taken.


In [ ]:
%%bash
source activate llava

In [ ]:
!conda install -y -c nvidia cuda-compiler

Channels:
 - nvidia
 - conda-forge
Platform: linux-64
Solving environment: / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cuda-compiler


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    cuda-compiler-12.4.1       |                0           2 KB  nvidia
    cuda-cuobjdump-12.4.127    |                0         302 KB  nvidia
    cuda-cuxxfilt-12.4.127     |                0         296 KB  nvidia
    cuda-nvcc-12.4.131         |                0        62.6 MB  nvidia
    cuda-nvprune-12.4.127      |                0          66 KB  nvidia
    ----------------------

In [ ]:
!conda list |grep nvidia

cuda-compiler             12.4.1                        0    nvidia
cuda-cuobjdump            12.4.127                      0    nvidia
cuda-cuxxfilt             12.4.127                      0    nvidia
cuda-nvcc                 12.4.131                      0    nvidia
cuda-nvprune              12.4.127                      0    nvidia


In [ ]:
!pip install pre-commit==3.0.2

# Install package locally
!pip install --upgrade pip  # enable PEP 660 support
!cd finetune_LLaVA/
!pip install -e .

# Install additional packages for training
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation


Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
  Using cached flash_attn-2.5.8.tar.gz (2.5 MB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 6.1 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.8-cp310-cp310-linux_x86_64.whl size=121687847 sha256=7544f11c6e8f86ed1b737b01bbe4fb1ccfad155dea5ff2d287699d04e6728459
  Stored in directory: /root/.cache/pip/wheels/9b/5b/2b/dea8af4e954161c49ef1941938afcd91bb93689371ed12a226
Successfully built flash-attn


In [ ]:
!pip install pre-commit==3.0.2

# Install package locally
!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .

# Install additional packages for training
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation


Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
